In [1]:
import pyspark

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/02 20:34:50 WARN Utils: Your hostname, boogie-notepad.local resolves to a loopback address: 127.0.0.1; using 192.168.1.147 instead (on interface en0)
23/03/02 20:34:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/02 20:34:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [21]:
import os

for color in ['green', 'yellow']:
    for year in ['2020', '2021']:
        for month in range(1,13):
            path_source = f'data/raw/{color}/{year}/{month:02d}/'
            if not os.path.exists(path_source):
                continue
            
            df = spark.read \
                .option("inferSchema", "true") \
                .option("header", "true") \
                .csv(f'data/raw/{color}/{year}/{month:02d}/')

            df.repartition(4) \
                .write.parquet(f'data/pq/{color}/{year}/{month:02d}/')

In [22]:
!tree data

data
├── pq
│   ├── green
│   │   ├── 2020
│   │   │   ├── 01
│   │   │   │   ├── _SUCCESS
│   │   │   │   ├── part-00000-180ee1ea-2cc2-496a-a750-52322c4e7468-c000.snappy.parquet
│   │   │   │   ├── part-00001-180ee1ea-2cc2-496a-a750-52322c4e7468-c000.snappy.parquet
│   │   │   │   ├── part-00002-180ee1ea-2cc2-496a-a750-52322c4e7468-c000.snappy.parquet
│   │   │   │   └── part-00003-180ee1ea-2cc2-496a-a750-52322c4e7468-c000.snappy.parquet
│   │   │   ├── 02
│   │   │   │   ├── _SUCCESS
│   │   │   │   ├── part-00000-197d5557-503e-4c31-80ee-aa13ea2b320b-c000.snappy.parquet
│   │   │   │   ├── part-00001-197d5557-503e-4c31-80ee-aa13ea2b320b-c000.snappy.parquet
│   │   │   │   ├── part-00002-197d5557-503e-4c31-80ee-aa13ea2b320b-c000.snappy.parquet
│   │   │   │   └── part-00003-197d5557-503e-4c31-80ee-aa13ea2b320b-c000.snappy.parquet
│   │   │   ├── 03
│   │   │   │   ├── _SUCCESS
│   │   │   │   ├── part-00000-e7a70529-5c4b-40c4-a406-b188fb497248-c000.snappy.parquet
│   │   │   │   ├── p

In [23]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [25]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [33]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime') \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')

In [36]:
df_green = df_green \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime') \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime')

In [39]:
common_columns = []

for green_col in df_green.columns:
    if green_col in df_yellow.columns:
            common_columns.append(green_col)

In [46]:
from pyspark.sql import functions as F

In [49]:
df_green_sel = df_green.select(common_columns) \
    .withColumn('service_type', F.lit('green'))

In [50]:
df_yellow_sel = df_yellow.select(common_columns) \
    .withColumn('service_type', F.lit('yellow'))

In [51]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [52]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [54]:
df_trips_data.createOrReplaceTempView('trips_data')

In [65]:
df_result = spark.sql(""" 
    select
    PULocationID as revenue_zone,
    date_trunc('month', pickup_datetime) as revenue_month, 
    service_type,  
    sum(fare_amount) as revenue_monthly_fare,
    sum(extra) as revenue_monthly_extra,
    sum(mta_tax) as revenue_monthly_mta_tax,
    sum(tip_amount) as revenue_monthly_tip_amount,
    sum(tolls_amount) as revenue_monthly_tolls_amount,
    sum(improvement_surcharge) as revenue_monthly_improvement_surcharge,
    sum(total_amount) as revenue_monthly_total_amount,
    sum(congestion_surcharge) as revenue_monthly_congestion_surcharge,
    count(1) as total_monthly_trips,
    avg(passenger_count) as avg_montly_passenger_count,
    avg(trip_distance) as avg_montly_trip_distance

    from trips_data
    group by 1,2,3
""")

In [66]:
df_result.write.parquet('data/report/revenue/')